### 행정안전부 보도자료 데이터 수집
- 정적페이지 데이터 수집 : html : bs, css-selector
- post 방식
- 여러 개의 페이지 데이터 수집

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### 1. 웹 서비스 분석 : url

In [4]:
page=3
url = 'https://mois.go.kr/frt/bbs/type010/commonSelectBoardList.do?bbsId=BBSMSTR_000000000008'
params = {
    "nttId": "0",
    "bbsTyCode": "BBST03",
    "bbsAttrbCode": "BBSA03",
    "authFlag": "Y",
    "pageIndex": page,
    "cal_url": "/sym/cal/EgovNormalCalPopup.do",
    "searchCnd": "0",
}

### 2. request(url) > response : html

In [7]:
response = requests.post(url, params)
response

<Response [200]>

### 3. html(str) > bs_obj.select(css-selector) > text > DataFrame

In [8]:
dom = BeautifulSoup(response.text, "html.parser")

In [10]:
# 게시글 리스트 데이터 선택 : 10개

In [35]:
elements = dom.select("#print_area > div.table_wrap.type_01 > form > table > tbody > tr")
len(element)

0

In [13]:
elements[0]

<tr>
<td class="res_hide">
								12931
							</td>
<td class="l">
<div class="wrap">
<a href="/frt/bbs/type010/commonSelectBoardArticle.do;jsessionid=B2UsCerEnVkMtHMWehvpj1H2.node20?bbsId=BBSMSTR_000000000008&amp;nttId=90455" onclick="javascript:fn_egov_inqire_notice('90455', 'BBSMSTR_000000000008'); return false;">사이버대학 성적·졸업증명서도 모바일 전자증명서로 발급한다 </a>
</div>
</td>
<td class="res_hide">
<img alt="첨부파일" src="/images/board/icon_file.gif;jsessionid=B2UsCerEnVkMtHMWehvpj1H2.node20"/>
</td>
<td class="res_hide">
								공공지능정책과
							</td>
<td>
								2022.02.15.
							</td>
<td class="res_hide">
								812
							</td>
</tr>

In [ ]:
# 각 데이터에서 필요한 정보 수집

In [19]:
element = elements[0]

data = {
    "no" : element.select("td")[0].text.strip(),
    "title" : element.select("td")[1].text.strip(),
    "writer" : element.select("td")[3].text.strip(),
    "date" : element.select("td")[4].text.strip(),
    "pv" : element.select("td")[5].text.strip(),
    "link" : "https://mois.go.kr" + element.select("td")[1].select_one('a').get('href'),
}

data

{'no': '12931',
 'title': '사이버대학 성적·졸업증명서도 모바일 전자증명서로 발급한다',
 'writer': '공공지능정책과',
 'date': '2022.02.15.',
 'pv': '812',
 'link': 'https://mois.go.kr/frt/bbs/type010/commonSelectBoardArticle.do;jsessionid=B2UsCerEnVkMtHMWehvpj1H2.node20?bbsId=BBSMSTR_000000000008&nttId=90455'}

In [23]:
# [{row1}, {row2}]

In [24]:
datas = []
for element in elements:
    datas.append({
        "no" : element.select("td")[0].text.strip(),
        "title" : element.select("td")[1].text.strip(),
        "writer" : element.select("td")[3].text.strip(),
        "date" : element.select("td")[4].text.strip(),
        "pv" : element.select("td")[5].text.strip(),
        "link" : "https://mois.go.kr" + element.select("td")[1].select_one('a').get('href'),
    })
    
datas

df = pd.DataFrame(datas)
df.head()

,no,title,writer,date,pv,link
0,12931,사이버대학 성적·졸업증명서도 모바일 전자증명서로 발급한다,공공지능정책과,2022.02.15.,812,https://mois.go.kr/frt/bbs/type010/commonSelec...
1,12930,"2022 지방행정혁신, 주민이 체감하는 확실한 변화 만든다",지역사회혁신정책과,2022.02.15.,677,https://mois.go.kr/frt/bbs/type010/commonSelec...
2,12929,적극행정을 일상적 공직문화로 자리매김하여 국민이 현장에서 체감하는 성과를 창출하겠습니다,지역사회혁신정책과,2022.02.15.,557,https://mois.go.kr/frt/bbs/type010/commonSelec...
3,12928,'가명정보 결합전문기관 실무협의회' 출범,공공데이터정책과,2022.02.14.,647,https://mois.go.kr/frt/bbs/type010/commonSelec...
4,12927,코로나19로 지친 국민들에게 지방세 지원을 강화한다,지방세정책과,2022.02.14.,945,https://mois.go.kr/frt/bbs/type010/commonSelec...


### 4. function : params : page

In [25]:
def mois(page):
    url = 'https://mois.go.kr/frt/bbs/type010/commonSelectBoardList.do?bbsId=BBSMSTR_000000000008'
    params = {
        "nttId": "0",
        "bbsTyCode": "BBST03",
        "bbsAttrbCode": "BBSA03",
        "authFlag": "Y",
        "pageIndex": page,
        "cal_url": "/sym/cal/EgovNormalCalPopup.do",
        "searchCnd": "0",
    }
    response = requests.post(url, params)
    dom = BeautifulSoup(response.text, "html.parser")
    elements = dom.select("#print_area > div.table_wrap.type_01 > form > table > tbody > tr")
    datas = []
    for element in elements:
        datas.append({
            "no" : element.select("td")[0].text.strip(),
            "title" : element.select("td")[1].text.strip(),
            "writer" : element.select("td")[3].text.strip(),
            "date" : element.select("td")[4].text.strip(),
            "pv" : element.select("td")[5].text.strip(),
            "link" : "https://mois.go.kr" + element.select("td")[1].select_one('a').get('href'),
        })
    return pd.DataFrame(datas)

In [26]:
page = 7
mois(page)

,no,title,writer,date,pv,link
0,12891,"정부청사, 중대재해처벌법 시행 앞두고 맞춤형 산업안전 강화 조치 시행",노사후생과,2022.01.25.,1438,https://mois.go.kr/frt/bbs/type010/commonSelec...
1,12890,"행안부, 현장에 강한 실전형 '지역 재난안전관리자' 양성",재난안전교육과,2022.01.25.,1266,https://mois.go.kr/frt/bbs/type010/commonSelec...
2,12889,"설명절 무료 개방 공공주차장 14,843개, '공유누리'에서 확인하세요",지역디지털서비스과,2022.01.25.,1380,https://mois.go.kr/frt/bbs/type010/commonSelec...
3,12888,"제2차 지방일괄이양 추진 12개 법안, 국무회의 의결",자치분권지원과,2022.01.25.,1381,https://mois.go.kr/frt/bbs/type010/commonSelec...
4,12887,국민 삶의 질을 높이는 성과를 창출한 정부혁신 우수기관 발표,혁신기획과,2022.01.25.,1305,https://mois.go.kr/frt/bbs/type010/commonSelec...
5,12886,자치분권위원회 김순은 위원장 연임,홍보담당관,2022.01.24.,1175,https://mois.go.kr/frt/bbs/type010/commonSelec...
6,12885,2022년 정부청사는 이렇게 달라집니다,관리총괄과,2022.01.24.,1363,https://mois.go.kr/frt/bbs/type010/commonSelec...
7,12884,적극행정으로 지역경제 살리고 쓰레기 매립지 수목원으로 바꿨다,지역사회혁신정책과,2022.01.24.,1310,https://mois.go.kr/frt/bbs/type010/commonSelec...
8,12883,"정부 과천청사관리소, 「상호 존중의 날」 행사 개최",관리과,2022.01.24.,1166,https://mois.go.kr/frt/bbs/type010/commonSelec...
9,12882,우수 재난안전제품 온라인으로 인증 신청하세요!,재난안전산업과,2022.01.23.,1615,https://mois.go.kr/frt/bbs/type010/commonSelec...


### 5. 여러페이지 수집

In [28]:
dfs = []
for page in range(1, 4):
    print(page, end=' ')
    dfs.append(mois(page))

1 2 3 

In [31]:
result_df = pd.concat(dfs, ignore_index=True)

In [32]:
result_df.tail(2)

,no,title,writer,date,pv,link
28,12923,스마트워크센터 밀집현상 해소를 위해 서울역 2호점 추가 개소,스마트행정기반과,2022.02.10.,1044,https://mois.go.kr/frt/bbs/type010/commonSelec...
29,12922,"2022년 지방규제, 지역활력 제고에 초점 맞추기로",지방규제혁신과,2022.02.10.,1122,https://mois.go.kr/frt/bbs/type010/commonSelec...
